In [1]:
from datasets import load_from_disk, Dataset
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from tqdm import tqdm, trange
import random
import torch
import torch.nn.functional as F
from retrieval_model import BertEncoder
from torch import nn
import pandas as pd
from torch.utils.data import (DataLoader, RandomSampler, TensorDataset, SequentialSampler)
from tqdm import tqdm, trange
import pickle

In [2]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)
np.random.seed(42)
random.seed(42)

In [3]:
# DPR model load
p_encoder = BertEncoder.from_pretrained("/opt/ml/mrc-level2-nlp-08/retrieval/p_encoder")
q_encoder = BertEncoder.from_pretrained("/opt/ml/mrc-level2-nlp-08/retrieval/q_encoder")
tokenizer =  AutoTokenizer.from_pretrained('klue/bert-base')

In [4]:
#train_dataset = load_from_disk("/opt/ml/data/train_dataset/train/")
#query_dataset = load_from_disk("../data/test_dataset/validation") # test query
#wiki_dataset = pd.read_json('../data/wikipedia_documents.json',orient='index') # wiki context
#wiki_dataset = pd.read_csv('/opt/ml/data/preprocess_wiki_doc.csv')
#train_dataset = load_from_disk("/opt/ml/data/train_dataset/new_validation/")
#origin_valid = load_from_disk("/opt/ml/data/train_dataset/validation/")

In [5]:
wiki_dataset = pd.read_csv('/opt/ml/data/preprocess_wiki_doc.csv')
query_dataset = load_from_disk("/opt/ml/data/train_dataset/validation/")

In [6]:
# num = 7
# print(train_dataset['question'][num])
# print(origin_valid['context'][num])
# print('---------------')
# print(train_dataset['context'][num])


In [7]:
#train_corpus = list(set([example for example in train_dataset['context']]))
wiki_corpus = list(set([example_wiki for example_wiki in wiki_dataset['text']]))
query = query_dataset['question']

In [8]:
print(len(query))
print(len(wiki_corpus))

240
55963


In [9]:
eval_batch_size = 32
def to_cuda(batch):
  return tuple(t.cuda() for t in batch)
if torch.cuda.is_available():
    p_encoder.cuda()
    q_encoder.cuda()

# Construt dataloader
train_p_seqs = tokenizer(wiki_corpus, padding="max_length", truncation=True, return_tensors='pt')
valid_dataset = TensorDataset(
    train_p_seqs["input_ids"],
    train_p_seqs["attention_mask"],
    train_p_seqs["token_type_ids"]
)
valid_sampler = SequentialSampler(valid_dataset)
valid_dataloader = DataLoader(
    valid_dataset,
    sampler=valid_sampler,
    batch_size=eval_batch_size
)

# Inference using the passage encoder to get dense embeddeings
p_embs = []

with torch.no_grad():

    epoch_iterator = tqdm(
        valid_dataloader,
        desc="Iteration",
        position=0,
        leave=True
    )
    p_encoder.eval()

    for _, batch in enumerate(epoch_iterator):
        batch = tuple(t.cuda() for t in batch)

        p_inputs = {
            "input_ids": batch[0],
            "attention_mask": batch[1],
            "token_type_ids": batch[2]
        }
        
        outputs = p_encoder(**p_inputs).to("cpu").numpy()
        p_embs.extend(outputs)

torch.cuda.empty_cache()

Iteration: 100%|██████████| 1749/1749 [09:40<00:00,  3.01it/s]


In [15]:
train_q_seqs = tokenizer(
    query,
    padding="max_length",
    truncation=True,
    return_tensors="pt"
)

query_dataset = TensorDataset(
    train_q_seqs["input_ids"],
    train_q_seqs["attention_mask"],
    train_q_seqs["token_type_ids"]
)

query_sampler = SequentialSampler(query_dataset)
query_dataloader = DataLoader(
    query_dataset,
    sampler=query_sampler,
    batch_size=eval_batch_size
)

q_embs = []

with torch.no_grad():

    epoch_iterator = tqdm(
        query_dataloader,
        desc="Iteration",
        position=0,
        leave=True
    )
    q_encoder.eval()

    for _, batch in enumerate(epoch_iterator):
        batch = tuple(t.cuda() for t in batch)

        q_inputs = {
            "input_ids": batch[0],
            "attention_mask": batch[1],
            "token_type_ids": batch[2]
        }
        
        outputs = q_encoder(**q_inputs).to("cpu").numpy()
        q_embs.extend(outputs)

torch.cuda.empty_cache()
print('done')

Iteration: 100%|██████████| 8/8 [00:02<00:00,  3.22it/s]

done


In [16]:
p_embs = np.array(p_embs)
q_embs = np.array(q_embs)

print(p_embs.shape)
print(q_embs.shape)

(55963, 768)
(240, 768)


In [17]:
if torch.cuda.is_available():
    p_embs_cuda = torch.Tensor(p_embs).to('cuda')
    q_embs_cuda = torch.Tensor(q_embs).to('cuda')

dot_prod_scores = torch.matmul(q_embs_cuda, torch.transpose(p_embs_cuda, 0, 1))
rank = torch.argsort(dot_prod_scores, dim=1, descending=True).squeeze()

In [18]:
dense_p_retrieval_result = {}
idx = 0
for i in tqdm(range(len(query))):
    p_list = []
    q = query[i]
    for j in range(100):
        p_list.append(wiki_corpus[rank[idx][j]])
    dense_p_retrieval_result[q] = p_list
    idx += 1

100%|██████████| 240/240 [00:00<00:00, 489.42it/s]


In [19]:
with open("/opt/ml/data/dense_valid_retrieval.bin", "wb") as file:
    pickle.dump(dense_p_retrieval_result,file)


# dense_n_retrieval_result = {}
# idx = 0
# for i in tqdm(range(len(query))):
#     p_list = []
#     q = query[i]
#     for j in range(10000,10004):
#         p_list.append(wiki_corpus[rank[idx][j]])
#     dense_n_retrieval_result[q] = p_list
#     idx += 1

In [20]:
dense_p_retrieval_result[query[0]][:5]

['2008년 소시에테 제네랄 은행의 선물딜러인 제롬 케르비엘에 의한 선물거래로 인해 49억 유로(71억 달러)의 손실이 발생하였다. 제롬은 은행 안에 비밀 사업체를 세우고 선물상품을 투자하다가 회사측에 71억 달러(한국돈으로 약 7조원)에 달하는 피해를 주었다. 이는 개인이 일으킨 금융사고로는 세계에서 가장 큰 규모의 사고로, 피해금액은 영국 베어링 은행을 외환파생상품 거래로 파산시킨 닉 리슨이 입힌 12억 달러보다도 큰 액수이다. 소시에테 제네랄 은행은 제롬이 다른 거래인의 명의를 도용해, 단독으로 선물상품 투자했다가 이 같은 손실을 냈다고 사고경위를 설명했으며, 제롬은 사표를 낸 상태이다. 다니엘 부통 최고 경영자도 회사를 그만두려고 했으나 이사회에서 반려되었다. 한편 소액주주를 포함한 100명이상의 주주들은 회사에 대해 소송을 할 예정이다.',
 '2007년 9월 26일, 껀터 대교 공사를 진행하던 도중에 지상 30m의 위치에 있던 교각이 붕괴되는 사고가 발생했다. 당시 250명의 노동자가 작업 중이었고, 다음날 27일에는 52명이 사망하고, 140명이 부상당했다고 보도되었다. 2008년 8월 11일에 사망자는 55명, 부상자는 79명으로 늘어났다. 10월 4일, 타이세이 건설 등의 JV를 비롯한 계약자들은 당장의 보상금으로 90억 동을 기부한다고 발표했다. (10억 동을 사망자에 대한 조위금, 부상자 위로금, 나머지 80억 동을 유족의 유아 70명이 18세가 될 때까지 양육비를 지급하기 위한 기금으로 조성) 이 프로젝트에 컨설턴트로 참여했던 니혼 코에이(日本工營)의 엔지니어는 2007년 6월 프로젝트 책임자에게 지보공 계통이 안전 계수를 채우지 못해 작업 조건이 매우 위험하다는 경고문을 보냈다. 다른 엔지니어도 2007년 1월 문서에서는 도움 버팀대 계통의 시공에 안전도를 확인을 위한 실험을 요구했다. 사고의 원인을 조사하기 위해 베트남 정부는 껀터 대교 붕괴 사고 조사 국가위원회를 설치했다. 컨설턴트의 경고 내용은 위원회 제2차 회의에서 거론되었고 

In [21]:
query_id = load_from_disk("../data/test_dataset/validation")

In [22]:
query_id

Dataset({
    features: ['id', 'question'],
    num_rows: 600
})

In [5]:
temp = {'a':[1],'b':[2]}

print(list(temp.keys()))
print(list(temp.values()))

['a', 'b']
[[1], [2]]


In [25]:
with open('/opt/ml/data/dense_train.bin', "rb") as file:
    dense_train_retrieval = pickle.load(file)

In [ ]:
print(query_dataset[''])
dense_train_retrieval[query[0]][0:5]

In [31]:
new_context = []
for i in tqdm(range(len(train_dataset))):
    train_context = train_dataset['context'][i]
    sim_context = dense_train_retrieval[query[i]] # context list
    cnt = 4
    sim_context_idx = 0
    sim_top_k = [train_context] # 정답 context를 제외한 top_k
    # add_context = ' '.join(sim_context)
    # sim_top_k.append(add_context)
    # new_context.append(' '.join(sim_top_k))
    while cnt != 0:
        if train_context != sim_context[sim_context_idx]:
            sim_top_k.append(sim_context[sim_context_idx])
            cnt -= 1
        sim_context_idx += 1
    add_sim_context = ' '.join(sim_top_k)
    new_context.append(add_sim_context)


100%|██████████| 3952/3952 [03:14<00:00, 20.30it/s]


In [32]:
train_df = train_dataset.to_pandas()
train_df['context'] = new_context
new_train_dataset = Dataset.from_pandas(train_df)
new_train_dataset.save_to_disk('/opt/ml/data/train_dataset/new_train_v2')

In [ ]:
# with open('/opt/ml/data/dense_query_wiki_retrieval.bin', "rb") as file:
#     pickle.dump(dense_p_retrieval_result, file)

In [46]:
train_df = train_dataset.to_pandas()
train_df['context'] = new_context

In [50]:
#train_origin_df = train_dataset.to_pandas()
# all_train = pd.concat([train_df, train_negative_df])
# all_train.reset_index(drop=True)
# all_train = all_train.drop(['__index_level_0__'],axis=1)
# new_train_dataset = Dataset.from_pandas(all_train)
# new_train_dataset.save_to_disk('/opt/ml/data/train_dataset/new_train')

In [28]:
# validation set
train_dataset = load_from_disk("/opt/ml/data/train_dataset/validation/")
query = train_dataset['question']

In [29]:
eval_batch_size = 16
train_q_seqs = tokenizer(
    query,
    padding="max_length",
    truncation=True,
    return_tensors="pt"
)

query_dataset = TensorDataset(
    train_q_seqs["input_ids"],
    train_q_seqs["attention_mask"],
    train_q_seqs["token_type_ids"]
)

query_sampler = SequentialSampler(query_dataset)
query_dataloader = DataLoader(
    query_dataset,
    sampler=query_sampler,
    batch_size=eval_batch_size
)

q_embs = []

with torch.no_grad():

    epoch_iterator = tqdm(
        query_dataloader,
        desc="Iteration",
        position=0,
        leave=True
    )
    q_encoder.eval()

    for _, batch in enumerate(epoch_iterator):
        batch = tuple(t.cuda() for t in batch)

        q_inputs = {
            "input_ids": batch[0],
            "attention_mask": batch[1],
            "token_type_ids": batch[2]
        }
        
        outputs = q_encoder(**q_inputs).to("cpu").numpy()
        q_embs.extend(outputs)

torch.cuda.empty_cache()

Iteration: 100%|██████████| 15/15 [00:02<00:00,  5.84it/s]


In [30]:
p_embs = np.array(p_embs)
q_embs = np.array(q_embs)

print(p_embs.shape)
print(q_embs.shape)

(3340, 768)
(240, 768)


In [31]:
if torch.cuda.is_available():
    p_embs_cuda = torch.Tensor(p_embs).to('cuda')
    q_embs_cuda = torch.Tensor(q_embs).to('cuda')

dot_prod_scores = torch.matmul(q_embs_cuda, torch.transpose(p_embs_cuda, 0, 1))
rank = torch.argsort(dot_prod_scores, dim=1, descending=True).squeeze()

In [32]:
dense_p_retrieval_result = {}
idx = 0
for i in tqdm(range(len(query))):
    p_list = []
    q = query[i]
    for j in range(10):
        p_list.append(train_corpus[rank[idx][j]])
    dense_p_retrieval_result[q] = p_list
    idx += 1

100%|██████████| 240/240 [00:00<00:00, 3473.40it/s]


In [33]:
new_context = []
for i in tqdm(range(len(train_dataset))):
    train_context = train_dataset['context'][i]
    sim_context = dense_p_retrieval_result[query[i]] # context list
    cnt = 4
    sim_context_idx = 0
    sim_top_k = [train_context] # 정답 context를 제외한 top_k
    # add_context = ' '.join(sim_context)
    # sim_top_k.append(add_context)
    # new_context.append(' '.join(sim_top_k))
    while cnt != 0:
        if train_context != sim_context[sim_context_idx]:
            sim_top_k.append(sim_context[sim_context_idx])
            cnt -= 1
        sim_context_idx += 1
    add_sim_context = ' '.join(sim_top_k)
    new_context.append(add_sim_context)

100%|██████████| 240/240 [00:00<00:00, 304.76it/s]


In [34]:
train_df = train_dataset.to_pandas()
train_df['context'] = new_context

In [36]:
print(train_df['question'][0])
train_df['context'][0]

처음으로 부실 경영인에 대한 보상 선고를 받은 회사는?


'순천여자고등학교 졸업, 1973년 이화여자대학교를 졸업하고 1975년 제17회 사법시험에 합격하여 판사로 임용되었고 대법원 재판연구관, 수원지법 부장판사, 사법연수원 교수, 특허법원 부장판사 등을 거쳐 능력을 인정받았다. 2003년 최종영 대법원장의 지명으로 헌법재판소 재판관을 역임하였다.\\n\\n경제민주화위원회(위원장 장하성이 소액주주들을 대표해 한보철강 부실대출에 책임이 있는 이철수 전 제일은행장 등 임원 4명을 상대로 제기한 손해배상청구소송에서 서울지방법원 민사합의17부는 1998년 7월 24일에 "한보철강에 부실 대출하여 은행에 막대한 손해를 끼친 점이 인정된다"며 "원고가 배상을 청구한 400억원 전액을 은행에 배상하라"고 하면서 부실 경영인에 대한 최초의 배상 판결을 했다. \\n\\n2004년 10월 신행정수도의건설을위한특별조치법 위헌 확인 소송에서 9인의 재판관 중 유일하게 각하 견해를 내었다. 소수의견에서 전효숙 재판관은 다수견해의 문제점을 지적하면서 관습헌법 법리를 부정하였다. 전효숙 재판관은 서울대학교 근대법학교육 백주년 기념관에서 열린 강연에서, 국회가 고도의 정치적인 사안을 정치로 풀기보다는 헌법재판소에 무조건 맡겨서 해결하려는 자세는 헌법재판소에게 부담스럽다며 소회를 밝힌 바 있다. 호르스트 제호퍼 주총리 밑에서 재무장관을 지냈으며, 주를 대변하는 연방 상원의원으로서 상원 재무위원회 소속이었다.\\n\\n재무장관 시절 유럽 연합 집행위원회의 일괄 지원을 받고자 부실 주 지원 대출은행인 바이에른LB의 재건을 감독하기도 했다. 2014년에는 바이에른LB를 압박하여 헝가리 측에 MKB 단위를 매각함으로서 20여년 간 20억 유로의 손실을 초래한 부실투자를 종식시키기도 했다. 2015년에는 한스 외르크 셸링 오스트리아 외무장관과 협상을 타결하여 하이포 알페아드리아뱅크 인터내셔널(케른텐주 지역 은행)의 붕괴에서 시작된 양측 정부의 법적 분쟁을 끝냈다. 양해 각서에 따르면 오스트리아는 바이에른주에 12억 3천만 유로를 지불하며, 모든 관련 소

In [37]:
new_train_dataset = Dataset.from_pandas(train_df)
new_train_dataset.save_to_disk('/opt/ml/data/train_dataset/new_validation_v2')
print('done')

done


In [17]:
len(new_train_dataset['context'])

240

In [100]:
new_reverse_context = []
new_reverse_answer = []
# len(train_dataset)
for i in tqdm(range(len(train_dataset))):
    train_context = train_dataset['context'][i]
    sim_context = dense_train_retrieval_result[query[i]] # context list


    cnt = 1
    sim_context_idx = 2
    sim_top_k = [] # 정답 context를 제외한 top_k
    temp_answer = {}

    add_start_idx = 0
    while cnt != 0:
        if train_context != sim_context[sim_context_idx]:
            sim_top_k.append(sim_context[sim_context_idx])
            add_start_idx += len(sim_context[sim_context_idx]) + 1 # 공백까지 포함
            cnt -= 1
        sim_context_idx += 1
    sim_top_k.append(train_context)
    add_sim_context = ' '.join(sim_top_k)
    # 시간 복잡도?
    temp_answer['answer_start'] = [train_dataset['answers'][i]['answer_start'][0] + add_start_idx]
    temp_answer['text'] = train_dataset['answers'][i]['text']
    new_reverse_answer.append(temp_answer)
    new_reverse_context.append(add_sim_context)


100%|██████████| 3952/3952 [15:34<00:00,  4.23it/s]


In [103]:
train_reverse_df = train_dataset.to_pandas()
train_reverse_df['context'] = new_reverse_context
train_reverse_df['answers'] = new_reverse_answer

In [104]:
train_reverse_df['context'][2][train_reverse_df['answers'][2]['answer_start'][0]]

'백'

In [107]:
new_reverse_negative_context = []
new_reverse_negative_answer = []
# len(train_dataset)
for i in tqdm(range(len(train_dataset))):
    train_context = train_dataset['context'][i]
    sim_context = dense_train_retrieval_result[query[i]] # context list


    cnt = 1
    sim_context_idx = -2
    sim_top_k = [] # 정답 context를 제외한 top_k
    temp_answer = {}

    add_start_idx = 0
    while cnt != 0:
        if train_context != sim_context[sim_context_idx]:
            sim_top_k.append(sim_context[sim_context_idx])
            add_start_idx += len(sim_context[sim_context_idx]) + 1 # 공백까지 포함
            cnt -= 1
        sim_context_idx -= 1
    sim_top_k.append(train_context)
    add_sim_context = ' '.join(sim_top_k)
    # 시간 복잡도?
    temp_answer['answer_start'] = [train_dataset['answers'][i]['answer_start'][0] + add_start_idx]
    temp_answer['text'] = train_dataset['answers'][i]['text']
    new_reverse_negative_answer.append(temp_answer)
    new_reverse_negative_context.append(add_sim_context)

100%|██████████| 3952/3952 [15:44<00:00,  4.18it/s]


In [108]:
train_reverse_negative_df = train_dataset.to_pandas()
train_reverse_negative_df['context'] = new_reverse_negative_context
train_reverse_negative_df['answers'] = new_reverse_negative_answer


In [110]:
train_origin_df = train_dataset.to_pandas()
all_train = pd.concat([train_origin_df, train_df, train_negative_df,train_reverse_df,train_reverse_negative_df])
all_train.reset_index(drop=True)
all_train = all_train.drop(['__index_level_0__'],axis=1)
new_train_dataset = Dataset.from_pandas(all_train)
new_train_dataset.save_to_disk('/opt/ml/data/train_dataset/new_train')

In [ ]:
all_train.reset_index(drop=True)

In [30]:
all_train = all_train.drop(['__index_level_0__'],axis=1)

In [31]:
new_train_dataset = Dataset.from_pandas(all_train)